# Zaawansowane operacje na tabeli CAS

In [ ]:
import swat
conn = swat.CAS('localhost', 5570)

In [ ]:
iris = conn.upload_file('../dane/iris.csv', casout=dict(name='IRIS',caslib='CASUSER', replace=True))

In [ ]:
cars = conn.upload_file('../dane/cars.csv', casout=dict(name='CARS',caslib='CASUSER', replace=True))

## Podstawowe statystyki

Za pomocą funkcji `discribe` można przywołać podstawowe statystyki.

In [ ]:
iris.describe()

Jeżeli zainteresowani jesteśmy konkretnymi percentylami:

In [ ]:
iris.describe(percentiles=[0.3, 0.8])

W describe możemy poprosić o analizę zmiennych tyu numerycznego albo znakowego.

In [ ]:
iris.describe(include='character')

In [ ]:
iris.describe(include='all')

In [ ]:
iris.describe(include=['numeric', 'character'])

Typy statystyk też można wyspecyfikować

In [ ]:
iris.describe(stats=['count', 'nmiss', 'sum', 'probt', 'freq'])

Wszystkie statysyki:

In [ ]:
iris.describe(stats='all')

Pojedyncze statystyki można uruchamiać niezależnie.

In [ ]:
iris.count()

In [ ]:
iris.mean()

In [ ]:
iris.probt()

itd.

## Zestaw akcji - simple

Posiadając załadowaną tabelę możemy wykonywać na niej akcje. Zapoznajmy się z zestawem podstawowych akcji o nazwie **simple**

In [ ]:
conn.simple?

### **simple.numRows** - liczba wierszy

In [ ]:
iris.numRows()

### **simple.distinct** - liczba unikalnych wartości

In [ ]:
iris.distinct()

Jako parametr możemy podać listę interesujących nas zmiennych. Bardzo wiele akcji ma możliwość ustawienia tego parametru.

In [ ]:
iris.distinct(inputs=['variety','sepal_width'])

### **simple.summary** - statystyki opisowe

In [ ]:
iris.summary()

Akcja **summary** może przyjąć parametry np. `subSet` do wyboru statystyk i `inputs` do wyboru zmiennych

In [ ]:
iris.summary(
    inputs=['sepal_length', 'sepal_width'],
    subSet=['MEAN', 'STD']
)

### **simple.freq** - tabele częstości

In [ ]:
cars.freq(inputs='type')

In [ ]:
cars.freq(inputs=['type','origin'])

### **simple.mdSummary** - wielowymiarowe statystyki opisowe

In [ ]:
cars.mdSummary(inputs='mpg_highway',
               sets=[
                   {'groupBy':{'Origin'}},
                   {'groupBy':{'Origin','Type'}}],
               subset=['MAX', 'MIN', 'MEAN', 'STD', 'STDERR', 'VAR', 'N'])

### Wykresy z danych CASTable

In [ ]:
iris.plot()

In [ ]:
iris[['sepal_length', 'sepal_width']].plot()

In [ ]:
iris[['sepal_length', 'sepal_width']].plot(kind='area')

In [ ]:
iris[['sepal_length', 'sepal_width']].plot.area()

In [ ]:
iris.plot.scatter('sepal_length', 'sepal_width')

In [ ]:
iris.plot.hist()

### Zapisywanie tabeli na dysk

In [ ]:
iris.save(name='irisout.csv', caslib='CASUSER', replace=True)

In [ ]:
iris.save(name='irisout.sashdat', caslib='CASUSER', replace=True)

Zapisane pliki w bibliotece możemy podejrzeć za pomocą akcji `table.fileInfo`

In [ ]:
conn.table.fileInfo()

In [ ]:
conn.table.tableInfo()

## Sortowanie, wybieranie danych oraz iterowanie

### Pobieranie danych posortowanych

In [ ]:
iris.sort_values(['sepal_length', 'sepal_width'])

In [ ]:
irisSorted = iris.sort_values(['sepal_length', 'sepal_width'])
irisSorted

In [ ]:
irisSorted.head(10)

In [ ]:
irisSorted.tail(5)

In [ ]:
irisSorted = iris.sort_values(['sepal_length', 'sepal_width'], ascending=[False, True])
irisSorted

In [ ]:
irisSorted.head(10)

### Iterowanie po kolumnach i wierszach

#### Podstawowe iterowanie

In [ ]:
for col in iris:
    print(col)

In [ ]:
for i, col in enumerate(iris):
    print(i, col)

In [ ]:
for col, dtype in zip(iris, iris.dtypes):
    print(col, dtype)

#### Iterowanie po kolumnach (CASColumn)

In [ ]:
for col, obj in iris.iteritems():
    print(col, obj)
    print('')

In [ ]:
for row in iris.iterrows():
    print(row)

In [ ]:
for row in iris.itertuples():
    print(row)

### Zaawansowane techniki wybierania danych

In [ ]:
expr = iris.petal_length > 6.5
expr

In [ ]:
expr.head()

In [ ]:
newIris = iris[expr]
newIris.head()

In [ ]:
newIris = iris[iris.petal_length > 6.5]
newIris.head()

In [ ]:
newIris2 = newIris[newIris.petal_width < 2.2]
newIris2.head()

In [ ]:
iris[(iris.petal_length > 6.5) & 
        (iris.petal_width < 2.2)]

In [ ]:
iris[(iris.petal_length + iris.petal_width) * 2 > 17.5].head()

In [ ]:
iris[iris.variety.str.upper().str.startswith('SET')]

In [ ]:
iris[iris.variety.str.upper().str.startswith('SET')].head()

## Żonglowanie danymi w locie

### Tworzenie kolumn wyliczanych

In [ ]:
iris['sepal_factor'] = ((iris.sepal_length + iris.sepal_width) * 2)
iris.head()

In [ ]:
iris['total_factor'] = iris.sepal_factor + iris.petal_width + iris.petal_length
iris.head()

In [ ]:
iris['names'] = 'sepal / petal'
iris.head()

In [ ]:
iris['cap_names'] = iris.names.str.title()
iris.head()

In [ ]:
iris

### Przetwarzanie w grupach

Specyfikacja gryp bezpośrednio na obiekcie CASTable.

In [ ]:
irisSorted.set_param('groupby', ['variety'])
irisSorted

In [ ]:
irisSorted.summary(subset=['MIN','MAX'])

In [ ]:
irisSorted.del_param('groupby')
irisSorted

Użycie metod DataFrame do ustawienia grupowania.

In [ ]:
grpiris = irisSorted.groupby(['variety'])
grpiris

In [ ]:
grpiris.summary(subset=['MIN','MAX'])

#### Grupowanie na kilka sposobów na raz

In [ ]:
grpmdsumm = irisSorted.mdsummary(sets=[dict(groupby=['sepal_length']),
                                dict(groupby=['petal_length'])])

In [ ]:
list(grpmdsumm.keys())

In [ ]:
grpmdsumm.get_set(1)

In [ ]:
grpmdsumm.get_set(1).concat_bygroups()

### Prosta regresja liniowa

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import swat
conn = swat.CAS('server.demo.sas.com', 8777, 'student', 'Metadata0')

In [ ]:
cars = conn.upload_file('../dane/cars.csv', casout=dict(name='CARS',caslib='CASUSER', replace=True))

In [ ]:
cars_local = cars.sample(n=20,seed=12345).fetch(to=20).Fetch
cars_local

In [ ]:
plt.scatter(cars_local.Weight, cars_local.MPG_Highway)

In [ ]:
model = cars.regression(alpha=0.05,
                        inputs='weight',
                        target='mpg_highway').Regression
model

In [ ]:
linear = model.Linear[0]
intercept = model.Intercept[0]

In [ ]:
cars.Weight.min()

In [ ]:
x = np.array([cars.Weight.min(),cars.Weight.max()])
x

In [ ]:
y = linear*x + intercept
y

In [ ]:
plt.scatter(cars_local.Weight, cars_local.MPG_Highway)
#cars.plot.scatter('Weight', 'MPG_Highway',sample_pct=0.10)
plt.plot(x, y, lw=3, c="orange", label="fit line between min and max")
plt.legend()
plt.show()

### Ćwiczenie 3

Wykonaj analizę danych w zbiorze class.csv. 

1\. Załaduj dane do CAS.

2\. Wyświetl histogramy dla zmiennych numerycznych.

3\. Dopasuj linię regresji dla zmiennych `waga = f(wzrost)`. Zanotuj postać modelu wraz ze współczynnikami. Wrysuj linię regresji w wykres punktowy.

4\. Zrób to samo dla jednostek metrycznych.

5*\. Zapoznaj się z dokumentacją i przykładami akcji `regression` z zestawu akcji `regression`.

http://documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.4&docsetId=casactstat&docsetTarget=casactstat_regression_toc.htm&locale=en

6*\. Spróbuj dopasować model regresji i wyświetlić wykresy dla `waga = f(wzrost, płeć)`.